# تحسين نماذج Open AI

هذا الدفتر مبني على الإرشادات الحالية الموجودة في [التخصيص الدقيق](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) من Open AI.

تحسين النماذج (Fine-tuning) يعزز أداء النماذج الأساسية لتطبيقك عن طريق إعادة تدريبها ببيانات وسياق إضافي مرتبط بحالة الاستخدام أو السيناريو المحدد. لاحظ أن تقنيات هندسة المطالبات مثل _التعلم بعدة أمثلة_ و _توليد الاسترجاع المعزز_ تتيح لك تحسين الطلب الافتراضي ببيانات ذات صلة لتحسين الجودة. مع ذلك، هذه الأساليب محدودة بحجم نافذة الرموز القصوى للنموذج الأساسي المستهدف.

مع التخصيص الدقيق، نحن فعلياً نعيد تدريب النموذج نفسه بالبيانات المطلوبة (مما يسمح لنا باستخدام عدد أكبر بكثير من الأمثلة مقارنة بما يمكن وضعه في نافذة الرموز القصوى) - وننشر نسخة _مخصصة_ من النموذج لم تعد بحاجة لتوفير أمثلة لها أثناء وقت الاستدلال. هذا لا يحسن فقط فعالية تصميم المطالبة لدينا (لدينا مرونة أكبر في استخدام نافذة الرموز لأشياء أخرى) بل قد يحسن أيضاً من تكاليفنا (عن طريق تقليل عدد الرموز التي نحتاج لإرسالها للنموذج أثناء وقت الاستدلال).

التخصيص الدقيق يتكون من 4 خطوات:
1. تجهيز بيانات التدريب ورفعها.
1. تشغيل مهمة التدريب للحصول على نموذج مخصص.
1. تقييم النموذج المخصص وتكرار العملية لتحسين الجودة.
1. نشر النموذج المخصص للاستخدام عند الرضا عن النتائج.

لاحظ أن ليس كل النماذج الأساسية تدعم التخصيص الدقيق - [راجع توثيق OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) لأحدث المعلومات. يمكنك أيضاً تخصيص نموذج سبق تخصيصه من قبل. في هذا الدرس، سنستخدم `gpt-35-turbo` كنموذج أساسي مستهدف للتخصيص الدقيق.

---


### الخطوة 1.1: جهّز مجموعة البيانات الخاصة بك

لنقم ببناء روبوت دردشة يساعدك على فهم الجدول الدوري للعناصر من خلال الإجابة على الأسئلة حول أي عنصر باستخدام قصيدة ليمريك. في هذا الدرس البسيط، سننشئ فقط مجموعة بيانات لتدريب النموذج مع بعض الأمثلة التي توضح الشكل المتوقع للبيانات. في الاستخدام الواقعي، ستحتاج إلى إنشاء مجموعة بيانات تحتوي على أمثلة أكثر بكثير. قد تتمكن أيضًا من استخدام مجموعة بيانات مفتوحة (في مجال تطبيقك) إذا كانت متوفرة، وإعادة تنسيقها لتناسب عملية التخصيص الدقيق.

بما أننا نركز على `gpt-35-turbo` ونبحث عن إجابة من دورة واحدة (إكمال دردشة)، يمكننا إنشاء أمثلة باستخدام [هذا التنسيق المقترح](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) الذي يعكس متطلبات إكمال الدردشة في OpenAI. إذا كنت تتوقع محتوى حواري متعدد الدورات، ستستخدم [تنسيق الأمثلة متعدد الدورات](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) والذي يتضمن معلمة `weight` لتحديد أي الرسائل يجب استخدامها (أو تجاهلها) أثناء عملية التخصيص الدقيق.

سنستخدم هنا التنسيق الأبسط لدورة واحدة في هذا الدرس. البيانات ستكون بصيغة [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) مع سجل واحد في كل سطر، وكل سجل عبارة عن كائن بتنسيق JSON. المقطع أدناه يعرض سجلين كمثال - راجع [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) لمجموعة الأمثلة الكاملة (10 أمثلة) التي سنستخدمها في درس التخصيص الدقيق. **ملاحظة:** يجب أن يكون كل سجل _مكتوبًا_ في سطر واحد فقط (وليس مقسمًا عبر عدة أسطر كما هو معتاد في ملفات JSON المنسقة).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

في الاستخدام الواقعي ستحتاج إلى مجموعة أمثلة أكبر بكثير للحصول على نتائج جيدة - التوازن سيكون بين جودة الإجابات والوقت/التكلفة لعملية التخصيص الدقيق. نحن نستخدم مجموعة صغيرة حتى نتمكن من إكمال التخصيص بسرعة لشرح العملية. راجع [هذا المثال من OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) لدرس تخصيص دقيق أكثر تعقيدًا.


### الخطوة 1.2 رفع مجموعة البيانات الخاصة بك

قم برفع البيانات باستخدام واجهة برمجة التطبيقات للملفات (Files API) [كما هو موضح هنا](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). لاحظ أنه لكي تتمكن من تشغيل هذا الكود، يجب أن تكون قد قمت بالخطوات التالية أولاً:
 - تثبيت حزمة بايثون `openai` (تأكد من استخدام إصدار >=0.28.0 للحصول على أحدث الميزات)
 - تعيين متغير البيئة `OPENAI_API_KEY` إلى مفتاح واجهة برمجة التطبيقات الخاص بك من OpenAI
للمزيد من المعلومات، راجع [دليل الإعداد](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) المقدم للدورة.

الآن، شغّل الكود لإنشاء ملف للرفع من ملف JSONL المحلي الخاص بك.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### الخطوة 2.1: إنشاء مهمة التخصيص باستخدام SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### الخطوة 2.2: التحقق من حالة المهمة

إليك بعض الأمور التي يمكنك القيام بها باستخدام واجهة برمجة التطبيقات `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - عرض آخر n مهام ضبط دقيق
- `client.fine_tuning.jobs.retrieve(<job_id>)` - الحصول على تفاصيل مهمة ضبط دقيق محددة
- `client.fine_tuning.jobs.cancel(<job_id>)` - إلغاء مهمة ضبط دقيق
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - عرض حتى n حدث من المهمة
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

أول خطوة في العملية هي _التحقق من ملف التدريب_ للتأكد من أن البيانات بصيغة صحيحة.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### الخطوة 2.3: تتبع الأحداث لمراقبة التقدم


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### الخطوة 2.4: عرض الحالة في لوحة تحكم OpenAI


يمكنك أيضًا عرض الحالة من خلال زيارة موقع OpenAI واستكشاف قسم _التخصيص الدقيق_ في المنصة. سيعرض لك ذلك حالة المهمة الحالية، كما يمكنك تتبع سجل تنفيذ المهام السابقة. في هذه الصورة، يمكنك أن ترى أن التنفيذ السابق فشل، بينما نجح التشغيل الثاني. للتوضيح، حدث هذا عندما استخدم التشغيل الأول ملف JSON يحتوي على سجلات بتنسيق غير صحيح - وبعد تصحيح ذلك، اكتمل التشغيل الثاني بنجاح وأصبح النموذج متاحًا للاستخدام.

![حالة مهمة التخصيص الدقيق](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ar.png)


يمكنك أيضًا عرض رسائل الحالة والقياسات من خلال التمرير للأسفل في لوحة المعلومات المرئية كما هو موضح:

| الرسائل | القياسات |
|:---|:---|
| ![الرسائل](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ar.png) |  ![القياسات](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ar.png)|


### الخطوة 3.1: استرجاع المعرف واختبار النموذج المحسن في الكود


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### الخطوة 3.2: تحميل واختبار النموذج المحسن في الـ Playground

يمكنك الآن اختبار النموذج الذي قمت بتحسينه بطريقتين. أولاً، يمكنك زيارة الـ Playground واستخدام قائمة النماذج المنسدلة لاختيار النموذج المحسن الجديد من بين الخيارات المتاحة. الخيار الآخر هو استخدام خيار "Playground" الظاهر في لوحة التحسين (انظر الصورة أعلاه)، والذي يفتح لك عرضًا _مقارنًا_ يعرض نسختي النموذج الأساسي والمحسن جنبًا إلى جنب لتقييم سريع.

![حالة مهمة التحسين](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ar.png)

كل ما عليك فعله هو إدخال سياق النظام المستخدم في بيانات التدريب الخاصة بك وكتابة سؤال الاختبار. ستلاحظ أن كلا الجانبين يتم تحديثهما بنفس السياق والسؤال. بعد تشغيل المقارنة، ستلاحظ الفرق في النتائج بينهما. _لاحظ كيف أن النموذج المحسن يعرض الإجابة بالتنسيق الذي قدمته في أمثلتك، بينما النموذج الأساسي يكتفي باتباع تعليمات النظام فقط_.

![حالة مهمة التحسين](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ar.png)

ستلاحظ أيضًا أن المقارنة تعرض عدد الرموز (tokens) لكل نموذج، والوقت المستغرق للاستدلال. **هذا المثال بالتحديد هو مثال بسيط يهدف إلى توضيح العملية فقط، ولا يعكس بيانات أو سيناريو واقعي**. قد تلاحظ أن كلا النموذجين يظهران نفس عدد الرموز (لأن سياق النظام وسؤال المستخدم متطابقان)، مع أن النموذج المحسن يستغرق وقتًا أطول في الاستدلال (لأنه نموذج مخصص).

في الحالات الواقعية، لن تستخدم مثالًا بسيطًا كهذا، بل ستقوم بتحسين النموذج باستخدام بيانات حقيقية (مثل كتالوج المنتجات لخدمة العملاء)، حيث ستكون جودة الإجابة أكثر وضوحًا. في _هذا_ السياق، الحصول على جودة إجابة مماثلة باستخدام النموذج الأساسي سيتطلب هندسة موجهات مخصصة أكثر، مما سيزيد من استهلاك الرموز وربما الوقت اللازم للاستدلال. _لتجربة ذلك، اطلع على أمثلة التحسين في OpenAI Cookbook للبدء._



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
